In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm


In [ ]:
# 1. Cấu hình thông số 

In [ ]:
# 2. Load dữ liệu văn bản đã được làm sạch

In [ ]:
# 3. Chuẩn bị mô hình BERT

In [ ]:
# 4. Sinh vector đặc trưng (embeddings) cho toàn bộ văn bản

In [ ]:
# 5. Huấn luyện mô hình Machine Learning (Train Logistic Regression và Random Forest)

In [ ]:
# 6. Đánh giá kết quả (Tính Accuracy và F1-score)

In [ ]:
# 7. Lưu kết quả huấn luyện và lưu kết quả thành results_bert.json